# KSB Checker

## Set up

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

import UI.dialogs as ui

In [ ]:
# Edit the template to modify the way the model evaluates the text.
from prompt import TEMPLATE

In [ ]:
# Choose which apprenticeship standard to evaluate the portfolio against.
from l3_digital_support_technician import criteria
#from l4_software_developer import criteria

### Choose the AI model

In [ ]:
#MODEL = "llama3.2"  # Quicker for testing
MODEL = "gpt-oss"  # Better for evaluation
#MODEL = "deepseek-r1:32b"  # Deepseek alternative

In [ ]:
model = OllamaLLM(model=MODEL)

## Get the portfolio text

In [ ]:
portfolio_filename = ui.file_to_open("txt")

In [ ]:
with open(portfolio_filename, "r", encoding="utf-8") as f:
    portfolio_text = f.read()

## Evaluate the portfolio

In [ ]:
# Construct the prompt using the template to add the portfolio text
# and KSBs from the apprenticeship standard.
prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [ ]:
chain = prompt | model

In [ ]:
evaluation_text = ""
evaluation_filename = ui.file_to_save("md")

In [ ]:
for heading, criteria_text in criteria.items():
    print(f"Evaluating: {heading}")
    response = chain.invoke(
        {
            "portfolio_text": f"{portfolio_text}",
            "ksbs": f"{criteria_text}",
        }
    )
    evaluation_text += f"## {heading}\n\n{response}\n\n"


### Save the evaluation

In [ ]:
if evaluation_filename:
    with open(evaluation_filename, 'w', encoding='utf-8') as f:
        f.write(evaluation_text)